<a href="https://colab.research.google.com/github/Colsai/small_projects/blob/main/Workplan_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_experimental

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 1.7 MB/s eta 0:00:00


In [ ]:
import sqlite3
import tempfile
import urllib.request
import zipfile

import pandas as pd
from langchain_experimental.sql import SQLDatabaseChain
from langchain import LLMChain, OpenAI, SQLDatabase
from langchain.agents import (AgentExecutor, Tool, ZeroShotAgent,
                              initialize_agent, load_tools)

In [ ]:
os.environ["OPENAI_API_KEY"] = ''

In [ ]:
os.chdir('/content/drive/MyDrive/OIG_Work_Langchain')

In [ ]:
df = pd.read_html('https://oig.hhs.gov/reports-and-publications/workplan/active-item-table.asp')[0][0:-1]
df = df.reset_index()

In [ ]:
[(col, type(col)) for col in df.columns]

[('index', str),
 ('Announced or Revised', str),
 ('Agency', str),
 ('Title', str),
 ('Component', str),
 ('Report Number(s)', str),
 ('Expected Issue Date (FY)', str),
 ('Summary', str),
 ('Tags', str)]

In [ ]:
con = sqlite3.connect("wp.db")
cur = con.cursor()
cur.execute(f"""
CREATE TABLE IF NOT EXISTS WORKPLAN(
    idx int primary key,
    announced varchar,
    agency varchar,
    component varchar,
    reportnumber varchar,
    efy varchar,
    summary varchar,
    tags varchar);
""")

In [ ]:
df.to_sql('WORKPLAN', con, if_exists = "replace", index = False)
con.commit()
con.close()

In [ ]:
db = SQLDatabase.from_uri("sqlite:///wp.db")
llm = OpenAI(temperature=0)
db_chain = SQLDatabaseChain(
    llm=llm,
    database=db,
    verbose=True,  # Show its work
    return_direct=True,  # Return the results without sending back to the LLM
)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/sql/base.py:66: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [ ]:
db_chain("How many reports are there?")



> Entering new SQLDatabaseChain chain...
How many reports are there?
SQLQuery:SELECT COUNT(*) FROM "WORKPLAN";
SQLResult: [(240,)]
> Finished chain.


{'query': 'How many reports are there?', 'result': '[(240,)]'}

In [ ]:
db_chain("How many different agencies are represented?")



> Entering new SQLDatabaseChain chain...
How many different agencies are represented?
SQLQuery:SELECT COUNT(DISTINCT "Agency") FROM "WORKPLAN";
SQLResult: [(26,)]
> Finished chain.


{'query': 'How many different agencies are represented?', 'result': '[(26,)]'}

In [ ]:
db_chain("Wnich work plan was announced or revised first?")



> Entering new SQLDatabaseChain chain...
Wnich work plan was announced or revised first?
SQLQuery:SELECT "Title", "Announced or Revised" FROM "WORKPLAN" ORDER BY "Announced or Revised" ASC LIMIT 1;
SQLResult: [('Medicaid Rehabilitation Services Made by Community Residence Providers', 'April 2022')]
> Finished chain.


{'query': 'Wnich work plan was announced or revised first?',
 'result': "[('Medicaid Rehabilitation Services Made by Community Residence Providers', 'April 2022')]"}

In [ ]:
db_chain("From their summaries, which work plans are about Medicare Advantage? Tell me their titles.")



> Entering new SQLDatabaseChain chain...
From their summaries, which work plans are about Medicare Advantage? Tell me their titles.
SQLQuery:SELECT "Title" FROM "WORKPLAN" WHERE "Summary" LIKE '%Medicare%' LIMIT 5;
SQLResult: [('Maintaining Buprenorphine Treatment for Medicare Enrollees With Opioid Use Disorder',), ('Medicare Part B Payments for Over-the-Counter COVID-19 Tests During the PHE Demonstration',), ('Medicare Part C High-Risk Diagnosis Codes Tool Kit',), ("CMS May Make Increased Payments to MA Organizations for Diagnoses That Were Reported on Physicians' Claims But Were Not Confirmed on a Concurrent Inpatient Stay",), ('Medicare Advantage Payments Generated by Health Risk Assessments for 2022',)]
> Finished chain.


{'query': 'From their summaries, which work plans are about Medicare Advantage? Tell me their titles.',
 'result': '[(\'Maintaining Buprenorphine Treatment for Medicare Enrollees With Opioid Use Disorder\',), (\'Medicare Part B Payments for Over-the-Counter COVID-19 Tests During the PHE Demonstration\',), (\'Medicare Part C High-Risk Diagnosis Codes Tool Kit\',), ("CMS May Make Increased Payments to MA Organizations for Diagnoses That Were Reported on Physicians\' Claims But Were Not Confirmed on a Concurrent Inpatient Stay",), (\'Medicare Advantage Payments Generated by Health Risk Assessments for 2022\',)]'}